In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [2]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [3]:
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [25]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(x_train)

In [26]:
model_cnn = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),

    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

c:\Users\R.PRAVEEN KUMAR\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model_cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)

model_cnn.fit(datagen.flow(x_train, y_train, batch_size=64), epochs=30, 
              validation_data=(x_test, y_test),callbacks=[lr_scheduler])

c:\Users\R.PRAVEEN KUMAR\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 48s 45ms/step - accuracy: 0.7863 - loss: 0.7106 - val_accuracy: 0.9882 - val_loss: 0.0396 - learning_rate: 0.0010
Epoch 2/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 45s 48ms/step - accuracy: 0.9565 - loss: 0.1442 - val_accuracy: 0.9899 - val_loss: 0.0310 - learning_rate: 0.0010
Epoch 3/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 81s 86ms/step - accuracy: 0.9676 - loss: 0.1075 - val_accuracy: 0.9928 - val_loss: 0.0256 - learning_rate: 0.0010
Epoch 4/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 44s 47ms/step - accuracy: 0.9722 - loss: 0.0910 - val_accuracy: 0.9930 - val_loss: 0.0217 - learning_rate: 0.0010
Epoch 5/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 46s 49ms/step - accuracy: 0.9743 - loss: 0.0847 - val_accuracy: 0.9925 - val_loss: 0.0230 - learning_rate: 0.0010
Epoch 6/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 42ms/step - accuracy: 0.9764 - loss: 0.0757 - val_accuracy: 0.9931 - val_loss: 0.0189 - learning_rate: 0.0010
Epoch 7/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 37s 39ms/step - accuracy: 0.9794 - l

In [6]:
test_loss, test_acc = model_cnn.evaluate(x_test, y_test)
print(f"CNN Test Accuracy : {test_acc}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9871 - loss: 0.0566
CNN Test Accuracy : 0.9905999898910522


In [ ]:
# model_cnn.save("digit_classifier.h5")

# Multidigit

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def preprocess_image(img) :
    
    thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

    # Draw bounding boxes to check detection
    img_copy = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show detected digits with bounding boxes
    plt.imshow(img_copy)
    plt.title("Detected Digits")
    plt.show()
    
    digit_images = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        digit = thresh[y:y+h, x:x+w]

        # Resize to match CNN input (28x28)
        digit = cv2.resize(digit, (28, 28))

        digit = digit / 255.0
        digit = np.expand_dims(digit, axis=-1)

        digit_images.append(digit)

    # Convert to numpy array
    digit_images = np.array(digit_images)

    print(f"Extracted {len(digit_images)} digits")    
    return digit_images


In [ ]:
def predict_number(digit_images) :
    
    # model = load_model("digit_classifier.h5")
    
    predictions = [np.argmax(model_cnn.predict(d.reshape(1, 28, 28, 1))) for d in digit_images]
    predicted_number = "".join(map(str, predictions))

    print(f"Predicted Number: {predicted_number}")

In [ ]:
image_path = "png.png"
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

digit_images = preprocess_image(img)

predict_number(digit_images)